In [2]:
!pip install numpy
!pip install pandas
!pip install BeautifulSoup
!pip install requests

folder_result = './result'

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^

***Вариант №36***

Задание №1

In [2]:
import os
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


def parse_html_to_json(folder, folder_result, fileName):
    data = []
    for filename in os.listdir(folder):
        if filename.endswith(".html"):
            with open(os.path.join(folder, filename), "r", encoding="utf-8") as file:
                soup = BeautifulSoup(file, "html.parser")
                [address, index] = (
                    soup.find("p", class_="address-p")
                    .text.split("Улица:")[1]
                    .split("Индекс:")
                )
                item = {
                    "fileName": filename,
                    "building": soup.find("h1", class_="title")
                    .text.split("Строение:")[1]
                    .strip(),
                    "address": address.strip(),
                    "index": int(index.strip()),
                    "floors": int(
                        soup.find("span", class_="floors")
                        .text.split("Этажи:")[1]
                        .strip()
                    ),
                    "year": int(
                        soup.find("span", class_="year")
                        .text.split("Построено в")[1]
                        .strip()
                    ),
                    "rating": float(
                        soup.find("div", class_="build-wrapper")
                        .find_all("div")[-1]
                        .find_all("span")[0]
                        .text.split("Рейтинг:")[1]
                        .strip()
                    ),
                    "views": int(
                        soup.find("div", class_="build-wrapper")
                        .find_all("div")[-1]
                        .find_all("span")[-1]
                        .text.split("Просмотры:")[1]
                        .strip()
                    ),
                    "isParking": (
                        True
                        if soup.find("div", class_="build-wrapper")
                        .find_all("div")[2]
                        .find_all("span")[-1]
                        .text.split("Парковка:")[1]
                        .strip()
                        == "есть"
                        else False
                    ),
                }
                data.append(item)

    # Сортировка по ключу "year"
    data = sorted(data, reverse=True, key=lambda x: x["year"])

    columnViews = "views"
    columnIsParking = "isParking"
    df = pd.DataFrame(data)

    # Статистические характеристики по колонке "views"
    views_sum = df[columnViews].sum()
    views_min = df[columnViews].min()
    views_max = df[columnViews].max()
    views_mean = df[columnViews].mean()
    views_median = df[columnViews].median()

    # Частота меток по колонке "isParking"
    is_parking_counts = df[columnIsParking].value_counts().to_dict()

    statistical_characteristics_views = {
        "views_sum": views_sum,
        "views_min": views_min,
        "views_max": views_max,
        "views_mean": views_mean,
        "views_median": views_median,
    }

    with open(
        os.path.join(folder_result, fileName), "w", encoding="utf-8"
    ) as json_file:
        json.dump(
            {
                "is_parking_counts": is_parking_counts,
                "statistical_characteristics_views": statistical_characteristics_views,
                "dataset": data,
            },
            json_file,
            cls=NpEncoder,
            ensure_ascii=False,
            indent=4,
        )


folder = "./data/1"
fileName = "data_result№1.json"
parse_html_to_json(folder, folder_result, fileName)

Задание №2

In [3]:
import os
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


def parse_html_to_json(folder, folder_result, fileName):
    data = []
    for filename in os.listdir(folder):
        if filename.endswith(".html"):
            with open(os.path.join(folder, filename), "r", encoding="utf-8") as file:
                soup = BeautifulSoup(file, "html.parser")
                allHTMLItems = soup.findAll("div", class_="pad")
                for HTMLItem in allHTMLItems:
                    item = {
                        "filename": filename,
                        "title": HTMLItem.find("span").text.strip(),
                        "coins_card": int(
                            HTMLItem.find("strong").text.strip().split(" ")[2]
                        ),
                        "price": int(
                            HTMLItem.find("price").text.split("₽")[0].replace(" ", "")
                        ),
                        "matrix": (
                            HTMLItem.find("li", attrs={"type": "matrix"}).text.strip()
                            if HTMLItem.find("li", attrs={"type": "matrix"}) != None
                            else None
                        ),
                    }
                    data.append(item)

    # Сортировка по ключу "price"
    data = sorted(data, key=lambda x: x["price"])

    column_coins_card = "coins_card"
    columnIsParking = "matrix"
    df = pd.DataFrame(data)

    # Статистические характеристики по колонке "coins_card"
    coins_card_sum = df[column_coins_card].sum()
    coins_card_min = df[column_coins_card].min()
    coins_card_max = df[column_coins_card].max()
    coins_card_mean = df[column_coins_card].mean()
    coins_card_median = df[column_coins_card].median()

    # Частота меток по колонке "isParking"
    matrix_types = df[columnIsParking].value_counts().to_dict()

    statistical_characteristics_coins_card = {
        "coins_card_sum": coins_card_sum,
        "coins_card_min": coins_card_min,
        "coins_card_max": coins_card_max,
        "coins_card_mean": coins_card_mean,
        "coins_card_median": coins_card_median,
    }

    with open(
        os.path.join(folder_result, fileName), "w", encoding="utf-8"
    ) as json_file:
        json.dump(
            {
                "matrix_types": matrix_types,
                "statistical_characteristics_coins_card": statistical_characteristics_coins_card,
                "dataset": data,
            },
            json_file,
            cls=NpEncoder,
            ensure_ascii=False,
            indent=4,
        )


folder = "./data/2"
fileName = "data_result№2.json"
parse_html_to_json(folder, folder_result, fileName)

Задание №3

In [4]:
import xml.etree.ElementTree as ET
import os
import json
import pandas as pd
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


def parse_xml_to_json(folder, folder_result, fileName):
    data = []
    for filename in os.listdir(folder):
        if filename.endswith(".xml"):
            tree = ET.parse(os.path.join(folder, filename))
            root = tree.getroot()
            item = {
                "filename": filename,
                "name": root.find("name").text.strip(),
                "constellation": root.find("constellation").text.strip(),
                "spectral_class": root.find("spectral-class").text.strip(),
                "radius": int(root.find("radius").text.strip()),
                "rotation": float(root.find("rotation").text.split()[0]),
                "age": float(root.find("age").text.split()[0]),
                "distance": float(root.find("distance").text.split()[0]),
                "absolute_magnitude": float(root.find("absolute-magnitude").text.split()[0]),
            }
            data.append(item)

    # Сортировка по ключу "age"
    data = sorted(data, key=lambda x: x["age"])

    column_radius = "radius"
    column_constellation = "constellation"
    df = pd.DataFrame(data)

    # Статистические характеристики по колонке "radius"
    radius_sum = df[column_radius].sum()
    radius_min = df[column_radius].min()
    radius_max = df[column_radius].max()
    radius_mean = df[column_radius].mean()
    radius_median = df[column_radius].median()

    # Частота меток по колонке "constellation"
    constellation_counts = df[column_constellation].value_counts().to_dict()

    statistical_characteristics_radius = {
        "radius_sum": radius_sum,
        "radius_min": radius_min,
        "radius_max": radius_max,
        "radius_mean": radius_mean,
        "radius_median": radius_median,
    }

    with open(
        os.path.join(folder_result, fileName), "w", encoding="utf-8"
    ) as json_file:
        json.dump(
            {
                "constellation_counts": constellation_counts,
                "statistical_characteristics_radius": statistical_characteristics_radius,
                "dataset": data,
            },
            json_file,
            cls=NpEncoder,
            ensure_ascii=False,
            indent=4,
        )


folder = "./data/3"
fileName = "data_result№3.json"
parse_xml_to_json(folder, folder_result, fileName)

Задание №4

In [5]:
import xml.etree.ElementTree as ET
import os
import json
import pandas as pd
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


def parse_xml_to_json(folder, folder_result, fileName):
    data = []
    for filename in os.listdir(folder):
        if filename.endswith(".xml"):
            tree = ET.parse(os.path.join(folder, filename))
            root = tree.getroot()
            for clothing in root.findall("clothing"):
                item = {}
                for child in clothing:
                    item[child.tag] = child.text.strip() if child.text else None
                data.append(item)

    # Сортировка по ключу "price"
    data = sorted(data, key=lambda x: int(x["price"]) if x["price"] else 0)

    column_price = "price"
    column_color = "color"
    df = pd.DataFrame(data)

    # Статистические характеристики по колонке "price"
    df[column_price] = pd.to_numeric(df[column_price], errors='coerce')
    price_sum = df[column_price].sum()
    price_min = df[column_price].min()
    price_max = df[column_price].max()
    price_mean = df[column_price].mean()
    price_median = df[column_price].median()

    # Частота меток по колонке "color"
    color_counts = df[column_color].value_counts().to_dict()

    statistical_characteristics_price = {
        "price_sum": price_sum,
        "price_min": price_min,
        "price_max": price_max,
        "price_mean": price_mean,
        "price_median": price_median,
    }

    with open(
        os.path.join(folder_result, fileName), "w", encoding="utf-8"
    ) as json_file:
        json.dump(
            {
                "color_counts": color_counts,
                "statistical_characteristics_price": statistical_characteristics_price,
                "dataset": data,
            },
            json_file,
            cls=NpEncoder,
            ensure_ascii=False,
            indent=4,
        )


folder = "./data/4"
fileName = "data_result№4.json"
parse_xml_to_json(folder, folder_result, fileName)

Задание №5

In [ ]:
import os
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


def parse_html_to_json(folder, folder_result, fileName):
    data = []
    for filename in os.listdir(folder):
        if filename.endswith(".html"):
            with open(os.path.join(folder, filename), "r", encoding="utf-8") as file:
                soup = BeautifulSoup(file, "html.parser")
                cars = soup.find_all("div", class_="ListingItem")
                for car in cars:
                    price = (
                        int(
                            car.find("div", class_="ListingItemPrice__content")
                            .text.strip()
                            .replace("\xa0", "")
                            .replace("от", "")
                            .replace("₽", "")
                        )
                        if car.find("div", class_="ListingItemPrice__content") != None
                        else 0
                    )
                    mileage = (
                        int(
                            car.find("div", class_="ListingItem__kmAge")
                            .text.strip()
                            .replace("\xa0", "")
                            .replace("км", "")
                            .replace("Новый", "0")
                        )
                        if car.find("div", class_="ListingItem__kmAge") != None
                        else None
                    )
                    item = {
                        "title": car.find(
                            "a", class_="ListingItemTitle__link"
                        ).text.strip(),
                        "price": price,
                        "year": int(
                            car.find("div", class_="ListingItem__year").text.strip()
                        ),
                        "mileage": mileage,
                        "type": car.findAll(
                            "div", class_="ListingItemTechSummaryDesktop__cell"
                        )[1].text.strip(),
                    }
                    data.append(item)

    # Сортировка по ключу "price"
    data = sorted(data, key=lambda x: x["price"])

    column_mileage = "mileage"
    column_location = "type"
    df = pd.DataFrame(data)

    # Статистические характеристики по колонке "price"
    mileage_sum = df[column_mileage].sum()
    mileage_min = df[column_mileage].min()
    mileage_max = df[column_mileage].max()
    mileage_mean = df[column_mileage].mean()
    mileage_median = df[column_mileage].median()

    statistical_characteristics_mileage = {
        "mileage_sum": mileage_sum,
        "mileage_min": mileage_min,
        "mileage_max": mileage_max,
        "mileage_mean": mileage_mean,
        "mileage_median": mileage_median,
    }

    # Частота меток по колонке "type"
    type_counts = df[column_location].value_counts().to_dict()

    with open(
        os.path.join(folder_result, fileName), "w", encoding="utf-8"
    ) as json_file:
        json.dump(
            {
                "type_counts": type_counts,
                "statistical_characteristics_mileage": statistical_characteristics_mileage,
                "dataset": data,
            },
            json_file,
            cls=NpEncoder,
            ensure_ascii=False,
            indent=4,
        )


folder = "./data/5"
fileName = "data_result№5.json"
parse_html_to_json(folder, folder_result, fileName)